In [54]:
# Packages and libraries
import pandas as pd
import xgboost as xgb
import numpy as np


from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split

from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold


import matplotlib.pyplot as plt

from matplotlib import pyplot as plt


def func_xg(df, model_name):
    for i in range(100):
        df = df.copy()

        if model_name == 'WO':
            X = df.drop(['COL_GRADE_AVG','Unnamed: 0','CR_S11','CC_S11','ENG_S11','CR_PRO','CC_PRO','ENG_PRO', 'STRATUM','GENDER','SCHOOL_NAT',	'SCHOOL_TYPE',	'MAT_S11'	,'BIO_S11'], axis=1).copy()
            y=df['COL_GRADE_AVG'].copy()
            X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=i, test_size=0.33)#, stratify=y)
            for_later_gender = X_test.drop(['HI_GRADE_AVG','QR_PRO',	'WC_PRO'], axis = 1)
            X_train = X_train.drop('GENDER_bin', axis = 1)
            X_test = X_test.drop('GENDER_bin', axis = 1)
            gender = for_later_gender.to_numpy()
        
        else:
            X = df.drop(['COL_GRADE_AVG','GENDER','Unnamed: 0','CR_PRO','CC_PRO','ENG_PRO'], axis=1).copy()                
            y=df['COL_GRADE_AVG'].copy()
            # for_later_gender = X_test.drop(['HI_GRADE_AVG','QR_PRO',	'WC_PRO'], axis = 1)
            
            X_encoded = pd.get_dummies(X,columns=['STRATUM','SCHOOL_TYPE','SCHOOL_NAT'])
            X_train, X_test, y_train, y_test = train_test_split(X_encoded,y,random_state=24, test_size=0.33)#, stratify=y)
            gender = X_test.GENDER_bin.to_numpy()
      


            
        # X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=i, test_size=0.33)#, stratify=y)
        # for_later_gender = X_test.drop(['HI_GRADE_AVG','QR_PRO',	'WC_PRO'], axis = 1)
        # X_train = X_train.drop('GENDER_bin', axis = 1)
        # X_test = X_test.drop('GENDER_bin', axis = 1)

        dfs = X_train.copy()
        for column in dfs.columns:
            try:
                dfs[column] = (dfs[column] - dfs[column].mean())/dfs[column].std()  
            except:
                pass
        X_train = dfs
        
        dft = X_test.copy()
        for column in dft.columns:
            try:
                dft[column] = (dft[column] - dft[column].mean())/dft[column].std()  
            except:
                pass

        X_test = dft

        final_model = xgb.XGBRegressor(seed=24, objective='reg:squarederror', max_depth=3, learning_rate=0.05, gamma=0,reg_lambda=15.0, scale_pos_weight=0.01)
        final_model.fit(X_train,y_train,verbose=True, eval_metric='rmse')

        y_pred_final = final_model.predict(X_test)
        
    





        index = X_test.index.to_numpy()
        

        
        preds = y_pred_final
        listen = list(map(list, zip(index,gender, preds)))



        samlet_preds_index_without_sensitive = pd.DataFrame(listen, columns=['Idx','Gender_bins','Preds'])
        samlet_preds_index_without_sensitive=samlet_preds_index_without_sensitive.sort_values(by=['Preds'],ascending=False)


        # from pathlib import Path
        # # src_path  = r"C:\Dokumenter\Dokumenter\UNI\7.semester\Bachelor\Bachelor_Gabi_Kat\src\Algoritmer\func_xg_wo_res\FileName.csv"
        
        # p = Path(src_path).parent.joinpath(f"samlet_preds_index_without_sensitive_ny{i}.csv")
        # df.to_csv(p,index=False)

        result = samlet_preds_index_without_sensitive.to_csv(f"func_xg_wo_res/samlet_preds_index_without_sensitive_ny{i}_{model_name}.csv")

    return result

In [55]:
df = pd.read_csv("df_sum_score_ex.csv")

func_xg(df, model_name='PLAIN')

In [ ]:
# Packages and libraries
import pandas as pd
import xgboost as xgb
import numpy as np

# Load data before standardization
df = pd.read_csv("df_sum_score_ex.csv")


X = df.drop(['COL_GRADE_AVG','GENDER','Unnamed: 0','CR_PRO','CC_PRO','ENG_PRO'], axis=1).copy()

y=df['COL_GRADE_AVG'].copy()


X_encoded = pd.get_dummies(X,columns=['STRATUM','SCHOOL_TYPE','SCHOOL_NAT'])
X_train, X_test, y_train, y_test = train_test_split(X_encoded,y,random_state=24, test_size=0.33)#, stratify=y)


#apply standardization to filtered original dataframe
dfs = X_train.copy()
for column in dfs.columns:
    try:
        dfs[column] = (dfs[column] - dfs[column].mean())/dfs[column].std()  
    except:
        pass

X_train = dfs
#apply standardization to filtered original dataframe

dft = X_test.copy()
for column in dft.columns:
    try:
        dft[column] = (dft[column] - dft[column].mean())/dft[column].std()  
    except:
        pass

X_test = dft
# Regressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt 
# XGBRegressor

model = xgb.XGBRegressor(objective='reg:squarederror')
model.fit(X_train, y_train)

# Prediction
y_pred_init = model.predict(X_test)
y_pred_init
# Mean squared error
# This is the average squared difference between the estimated values and the actual value.
# Kør disse herunder for at se om det er blevet bedre 
mse_init = mean_squared_error(y_test, y_pred_init)

# Root mean squared error
rmse_init= np.sqrt(mse_init)

print("INITIAL MSE: %.2f" % mse_init)
print("INITIAL RMSE: %.2f" % rmse_init)

# # Optimize parameters using cross-validation

# # XGBoost has a lot of hyper parameters which we can manually tune. These are e.g. max_depth of the tree, learning_rate, gamma which encourages pruning of the tree and reg_lambda which is the regularization parameter lambda. We are trying to find the optimal values for these parameters and hope that we can improve the accuracy in the prediction of the candidates college grades. 
# # Gridsearch cross validation 

# # Gridsearch CV will be run sequentially on subsets of different options for the parameters. 

# # 1st round
# param_grid_1 = {
#     'max_depth':[3,4,5],
#     'learning_rate':[0.01,0.05,0.1],
#     'gamma':[0,0.25,0.1],
#     'reg_lambda':[0,0.1,10.0],
#     'scale_pos_weight':[1,3,5]
# }
# from sklearn.model_selection import GridSearchCV


# # base_estimator_model = xgb.XGBRegressor()
# # clf = GridSearchCV(base_estimator_model, param_grid)
# # clf.fit(X_train, y_train)

# optimal_params = GridSearchCV(
#     estimator=xgb.XGBRegressor(
#         objective='reg:squarederror', seed=24, subsample=0.9),
#         param_grid=param_grid_1,
#         cv=5).fit(X_train,y_train)
          
# # optimal parameters 
# print(optimal_params.best_params_)


# ## Round 2 
# param_grid_2 = {
#     'max_depth':[2,3],
#     'learning_rate':[0.05],
#     'gamma':[0,0.01],
#     'reg_lambda':[10.0,15.0,20.0],
#     'scale_pos_weight':[0.01,0.5,1]
# }
# optimal_params = GridSearchCV(
#     estimator=xgb.XGBRegressor(
#         objective='reg:squarederror', seed=24, subsample=0.9),
#         param_grid=param_grid_2,
#         cv=5).fit(X_train,y_train)
# # optimal parameters 
# print(optimal_params.best_params_)


# ## Round 3 
# param_grid_3 = {
#     'max_depth':[3],
#     'learning_rate':[0.05],
#     'gamma':[0],
#     'reg_lambda':[10.0],
#     'scale_pos_weight':[1]
# }


final_model = xgb.XGBRegressor(seed=24, objective='reg:squarederror', max_depth=3, learning_rate=0.05, gamma=0,reg_lambda=10.0, scale_pos_weight=1)
final_model.fit(X_train,y_train,verbose=True, eval_metric='rmse')
# eval_metric default according to objective
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(final_model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

# force scores to be positive
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )

# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

# force scores to be positive
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )

# The model is tunes with grid search CV and evaluated using k-fold cross valdiation. The model is 
# evaluated on the training data and the average mean absolut error across the repeats of 10-fold cross-validtaion is reported. 
# There is some variation in the magnitude of errors as seen by theMAE. The average difference between 
# the predicted college garde and the observed college grade was 11.431 points in the grade. 

scores_final = cross_val_score(final_model, X_train, y_train, cv=10)
print("Mean cross-validation score: %.2f" % scores_final.mean())

scores = cross_val_score(model, X_train, y_train, cv=10)
print("Mean cross-validation score: %.2f" % scores.mean())
y_pred_final = final_model.predict(X_test)
y_pred_final
# Kør disse herunder for at se om det er blevet bedre 
mse_final = mean_squared_error(y_test, y_pred_final)
print("FINAL MSE: %.2f" % mse_final)

# Root mean squared error
rmse_final= np.sqrt(mse_final)
print("FINAL RMSE: %.2f" % rmse_final)

# from sklearn.metrics import average_precision_score
# map_final= average_precision_score(y_test, y_pred_final)
# print("FINAL MAP: %.2f" % map_final)


# Results
FOR RAW DATA VANILLA 
Before and after parameter optimization
- MSE: 219.97  (ændring på -6,44%)
- FINAL MSE: 205.79

FOR RAW DATA WITHOUT SENSITIVE 
- INITIAL MSE: 207.27
- FINAL MSE: 205.79

FOR STANDARDIZED DATA 
- INITIAL MSE: 0.42  (ændring på -4,76%)
- FINAL MSE: 0.40

- FINAL MSE: 0.40
- FINAL RMSE: 0.63

EX standardized data

Mean MAE: 11.055 (0.244)
Mean MAE: 11.703 (0.304)
Mean cross-validation score: 0.62
Mean cross-validation score: 0.57
FINAL MSE: 197.41
FINAL RMSE: 14.05


# plot decision tree

from xgboost import plot_tree
import matplotlib.pyplot as plt

from matplotlib import pyplot as plt
plt.figure(figsize=(20,20))



plot_tree(final_model,fontsize=10)
# plt.show()
plt.savefig('tree_high_dpi_ex', dpi=400)


# Reparring the index and predictions 
index = X_test.index.to_numpy()
gender = X_test.GENDER_bin.to_numpy()
gender[:7]


preds = y_pred_final # her har de stadig samme rækkefølge i forhold til index som til at starte med, de er nemlig ikke blevet sorteret endnu
listen = list(map(list, zip(index,gender, preds)))

for_later_gender = gender
for_later_gender = np.save('for_later_gender.npy', for_later_gender)    # .npy extension is added if not given


samlet_preds_plain_ex = pd.DataFrame(listen, columns=['Idx','Gender_bins','Preds'])


samlet_preds_plain_ex = samlet_preds_plain_ex.sort_values(by=['Preds'],ascending=False)
samlet_preds_plain_ex.to_csv("samlet_preds_plain_ex.csv")
samlet_preds_plain_ex 
# df = pd.read_csv("df_sum_score_ex.csv")
# dfs = df.copy()
# mean_col = []
# std_col = []
# col = []
# for column in dfs.columns:
#     try: 
#         mean_col.append(dfs[column].mean())
#         col.append(column)
#         std_col.append(dfs[column].std())
#     except:
#         pass 

# raw_mean_std = pd.DataFrame(
#     {'mean': mean_col,
#      'std': std_col,
#      'col': col
#     })

# #apply standardization to filtered original dataframe
# dfs_rev = samlet_preds_plain_ex.copy()
# dfs_rev
# sorte predictions med mergesort 
import numpy as np

pred_sorted = np.sort(y_pred_final, axis=- 1, kind='mergesort')[::-1]
pred_sorted



# Plotting
import matplotlib as mpl
def setup_mpl():
    mpl.rcParams["font.family"] = "Helvetica Neue"
    mpl.rcParams["font.size"] = 11
    mpl.rcParams["figure.figsize"] = (10,5)
    mpl.rcParams["figure.dpi"] = 200
    # mpl.rcParams["lines.linewidth"] = 1
setup_mpl()

x_ax = range(len(y_test))
y_test_sorted = np.sort(y_test, axis=- 1, kind='mergesort')[::-1]
plt.plot(x_ax, y_test_sorted, label="original")
plt.plot(x_ax, pred_sorted, label="predicted")
plt.title("College admissions test and predicted data")
plt.legend()
plt.show()
plt.savefig("plot_xgb_plain_ex.png")